In [2]:
# diagnostic.py
import torch
import json
from pathlib import Path

def check_model_structure(model_path):
    """Проверяет структуру сохраненной модели"""
    model_path = Path(model_path)
    
    # Проверяем конфигурацию
    config_file = model_path / "config.json"
    if config_file.exists():
        with open(config_file, 'r') as f:
            config = json.load(f)
        print(f"Конфигурация модели:")
        print(f"  model_type: {config.get('model_type', 'unknown')}")
        print(f"  d_model: {config.get('d_model', 'unknown')}")  # Размерность
        print(f"  d_ff: {config.get('d_ff', 'unknown')}")
        print(f"  num_layers: {config.get('num_layers', 'unknown')}")
    
    # Проверяем веса
    weights_file = model_path / "pytorch_model.bin"
    if weights_file.exists():
        state_dict = torch.load(weights_file, map_location='cpu')
        
        print(f"\nРазмеры ключевых слоев:")
        for key in list(state_dict.keys())[:10]:  # Первые 10 слоев
            if 'weight' in key and 'embed' not in key:
                shape = state_dict[key].shape
                print(f"  {key}: {shape}")
                
                # Определяем тип модели по размерности
                if len(shape) == 2:
                    if shape[0] == 768 or shape[1] == 768:
                        print("  → Похоже на t5-base (768 размерность)")
                    elif shape[0] == 512 or shape[1] == 512:
                        print("  → Похоже на t5-small (512 размерность)")

if __name__ == "__main__":
    check_model_structure("fine_tuned_t5_model")

Конфигурация модели:
  model_type: unknown
  d_model: unknown
  d_ff: unknown
  num_layers: unknown


In [1]:
"""
Скрипт для восстановления модели после обучения
"""

import torch
import json
from pathlib import Path
from transformers import T5Config, T5ForConditionalGeneration, T5Tokenizer
import numpy as np

class ModelRepair:
    def __init__(self, model_dir: str):
        self.model_dir = Path(model_dir)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def analyze_model(self):
        """Анализирует сохраненную модель"""
        print("="*60)
        print("АНАЛИЗ СОХРАНЕННОЙ МОДЕЛИ")
        print("="*60)
        
        # 1. Проверяем файлы
        print("\n1. Файлы в директории:")
        files = list(self.model_dir.glob("*"))
        for f in files:
            print(f"  ✓ {f.name}")
        
        # 2. Проверяем веса модели
        model_file = self.model_dir / "pytorch_model.bin"
        if model_file.exists():
            state_dict = torch.load(model_file, map_location="cpu")
            
            print("\n2. Анализ весов модели:")
            
            # Определяем размерность по весам
            for key in list(state_dict.keys()):
                if "encoder.block.0.layer.0.SelfAttention.q.weight" in key:
                    shape = state_dict[key].shape
                    print(f"  Ключ: {key}")
                    print(f"  Размер: {shape}")
                    
                    # Определяем тип модели
                    if shape == (768, 768):
                        print("  → Определено: T5-BASE (768 размерность)")
                        return "t5-base"
                    elif shape == (512, 512):
                        print("  → Определено: T5-SMALL (512 размерность)")
                        return "t5-small"
                    elif shape == (1024, 1024):
                        print("  → Определено: T5-LARGE (1024 размерность)")
                        return "t5-large"
            
            # Альтернативный способ определения
            print("\n3. Дополнительный анализ:")
            
            # Считаем количество блоков
            encoder_blocks = sum(1 for k in state_dict.keys() if "encoder.block" in k and ".layer.0." in k)
            decoder_blocks = sum(1 for k in state_dict.keys() if "decoder.block" in k and ".layer.0." in k)
            
            print(f"  Блоков в encoder: {encoder_blocks}")
            print(f"  Блоков в decoder: {decoder_blocks}")
            
            # Определяем по количеству блоков
            if encoder_blocks == 12 and decoder_blocks == 12:
                print("  → Вероятно: T5-BASE (12 блоков)")
                return "t5-base"
            elif encoder_blocks == 6 and decoder_blocks == 6:
                print("  → Вероятно: T5-SMALL (6 блоков)")
                return "t5-small"
        
        print("\n⚠ Не удалось определить тип модели автоматически")
        return None
    
    def create_config(self, model_type: str):
        """Создает правильный config.json"""
        
        configs = {
            "t5-small": {
                "d_model": 512,
                "d_ff": 2048,
                "d_kv": 64,
                "num_decoder_layers": 6,
                "num_heads": 8,
                "num_layers": 6,
                "num_layers": 6,
                "vocab_size": 32128,
            },
            "t5-base": {
                "d_model": 768,
                "d_ff": 3072,
                "d_kv": 64,
                "num_decoder_layers": 12,
                "num_heads": 12,
                "num_layers": 12,
                "vocab_size": 32128,
            },
            "t5-large": {
                "d_model": 1024,
                "d_ff": 4096,
                "d_kv": 64,
                "num_decoder_layers": 24,
                "num_heads": 16,
                "num_layers": 24,
                "vocab_size": 32128,
            }
        }
        
        if model_type not in configs:
            print(f"Неизвестный тип модели: {model_type}")
            return
        
        base_config = configs[model_type]
        
        config = T5Config(**base_config)
        config.architectures = ["T5ForConditionalGeneration"]
        config.model_type = "t5"
        config.is_encoder_decoder = True
        config.torch_dtype = "float32"
        
        # Сохраняем
        config.save_pretrained(self.model_dir)
        print(f"\n✅ Конфигурация для {model_type} создана")
    
    def repair_tokenizer(self):
        """Восстанавливает токенизатор"""
        print("\n4. Восстановление токенизатора...")
        
        # Пробуем разные модели
        model_names = [
            "t5-small",
            "t5-base", 
            "t5-large",
            "cointegrated/rut5-base"
        ]
        
        for model_name in model_names:
            try:
                tokenizer = T5Tokenizer.from_pretrained(model_name)
                tokenizer.save_pretrained(self.model_dir)
                print(f"  ✓ Токенизатор от {model_name} сохранен")
                return True
            except:
                continue
        
        print("  ✗ Не удалось восстановить токенизатор")
        return False
    
    def test_model_loading(self):
        """Тестирует загрузку модели"""
        print("\n5. Тестирование загрузки модели...")
        
        try:
            # Загружаем конфиг
            config = T5Config.from_pretrained(str(self.model_dir))
            
            # Создаем модель с правильной архитектурой
            model = T5ForConditionalGeneration(config)
            
            # Загружаем веса
            state_dict = torch.load(
                self.model_dir / "model.safetensors",
                map_location=self.device,
                weights_only=False
            )
            
            # Загружаем веса, игнорируя несоответствия
            model.load_state_dict(state_dict, strict=False)
            model.to(self.device)
            
            print("  ✓ Модель успешно загружена!")
            return True
            
        except Exception as e:
            print(f"  ✗ Ошибка: {e}")
            return False
    
    def repair(self):
        """Основной метод восстановления"""
        print("\n" + "="*60)
        print("ВОССТАНОВЛЕНИЕ МОДЕЛИ")
        print("="*60)
        
        # 1. Анализируем модель
        model_type = self.analyze_model()
        
        if not model_type:
            model_type = input("\nВведите тип модели вручную (t5-small/t5-base/t5-large): ")
        
        # 2. Создаем конфиг
        self.create_config(model_type)
        
        # 3. Восстанавливаем токенизатор
        self.repair_tokenizer()
        
        # 4. Тестируем
        success = self.test_model_loading()
        
        if success:
            print("\n" + "="*60)
            print("✅ МОДЕЛЬ ВОССТАНОВЛЕНА УСПЕШНО!")
            print("="*60)
        else:
            print("\n" + "="*60)
            print("⚠ ПРОБЛЕМЫ С ВОССТАНОВЛЕНИЕМ")
            print("="*60)
        
        return success

# Использование
if __name__ == "__main__":
    repair = ModelRepair("fine_tuned_t5_model")
    repair.repair()

c:\Users\loban\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



ВОССТАНОВЛЕНИЕ МОДЕЛИ
АНАЛИЗ СОХРАНЕННОЙ МОДЕЛИ

1. Файлы в директории:
  ✓ added_tokens.json
  ✓ all_results.json
  ✓ checkpoint-20
  ✓ config.json
  ✓ generation_config.json
  ✓ model.safetensors
  ✓ special_tokens_map.json
  ✓ spiece.model
  ✓ tokenizer_config.json
  ✓ training_args.bin
  ✓ train_results.json

⚠ Не удалось определить тип модели автоматически
Неизвестный тип модели: rut5-base

4. Восстановление токенизатора...


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  ✓ Токенизатор от t5-small сохранен

5. Тестирование загрузки модели...
  ✗ Ошибка: invalid load key, '`'.

⚠ ПРОБЛЕМЫ С ВОССТАНОВЛЕНИЕМ


In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import json

model_path = "./fine_tuned_t5_model"

try:
    # Попытка загрузки через Auto классы
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_path,
        torch_dtype=torch.float32,
        low_cpu_mem_usage=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
except RuntimeError as e:
    print(f"Ошибка при загрузке: {e}")
    print("Пробуем загрузить с ignore_mismatched_sizes=True")
    
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_path,
        ignore_mismatched_sizes=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)

# Чтение входного текста
with open("../rag_data/Pages/inputs/page_1_DHT3622X.txt", "r", encoding="utf-8") as f:
    input_text = f.read().strip()

# Токенизация
inputs = tokenizer(
    input_text,
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

# Параметры генерации
with open(f"{model_path}/config.json", "r") as f:
    gen_config = json.load(f)

# Генерация
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_beams=gen_config.get("num_beams", 4),
        temperature=gen_config.get("temperature", 1.0),
        top_p=gen_config.get("top_p", 0.9),
        top_k=gen_config.get("top_k", 50),
        max_length=gen_config.get("max_target_length", 512)
    )

# Декодирование
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Сохранение
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(generated_text)

print(f"Сгенерировано {len(generated_text)} символов")
print("Результат сохранён в output.txt")

`torch_dtype` is deprecated! Use `dtype` instead!


Ошибка при загрузке: Error(s) in loading state_dict for Linear:
	size mismatch for weight: copying a param with shape torch.Size([768, 768]) from checkpoint, the shape in current model is torch.Size([512, 512]).
Пробуем загрузить с ignore_mismatched_sizes=True


Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at ./fine_tuned_t5_model and are newly initialized: ['decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.1.layer.2.DenseReluDense.wi.weight', 'decoder.block.2.layer.2.DenseReluDense.wi.weight', 'decoder.block.3.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.2.DenseReluDense.wi.weight', 'decoder.block.5.layer.2.DenseReluDense.wi.weight', 'encoder.block.0.layer.1.DenseReluDense.wi.weight', 'encoder.block.1.layer.1.DenseReluDense.wi.weight', 'encoder.block.2.layer.1.DenseReluDense.wi.weight', 'encoder.block.3.layer.1.DenseReluDense.wi.weight', 'encoder.block.4.layer.1.DenseReluDense.wi.weight', 'encoder.block.5.layer.1.DenseReluDense.wi.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at ./fine_tuned_t5_model and are

ValueError: `decoder_start_token_id` or `bos_token_id` has to be defined for encoder-decoder generation.